In [3]:
import os
import random
import shutil
import json
from tqdm import tqdm

class DataSplitter:
    def __init__(self, data_path, output_path, split_ratio_train, split_ratio_val):
        self.data_path = data_path
        self.output_path = output_path
        self.split_ratio_train = split_ratio_train
        self.split_ratio_val = split_ratio_val
        self.coco_train = {
            "images": [],
            "annotations": [],
            "categories": []
        }
        self.coco_val = {
            "images": [],
            "annotations": [],
            "categories": []
        }
        self.annotation_id = 1

    def split_data(self):
        # 각 데이터 폴더를 생성합니다.
        train_path = os.path.join(self.output_path, "train")
        val_path = os.path.join(self.output_path, "val")
        annotations_train_path = os.path.join(self.output_path, "annotations", "train.json")
        annotations_val_path = os.path.join(self.output_path, "annotations", "val.json")
        
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "annotations"), exist_ok=True)

        # 이미지와 JSON 파일을 순서대로 가져와서 train, val 폴더로 복사하고 JSON 파일을 annotations 폴더에 매칭시켜 복사합니다.
        image_id = 1
        for root, dirs, files in tqdm(os.walk(self.data_path), desc="Splitting data"):
            for file in files:
                if file.endswith((".jpg", ".jpeg", ".png")):
                    img_path = os.path.join(root, file)
                    json_path = os.path.join(root.replace("img", "label"), file.split(".")[0] + ".json")

                    rand_val = random.random()
                    if rand_val < self.split_ratio_train:
                        dest_folder = train_path
                        coco_dataset = self.coco_train
                    else:
                        dest_folder = val_path
                        coco_dataset = self.coco_val

                    # 이미지 파일을 이동시킵니다.
                    os.makedirs(dest_folder, exist_ok=True)
                    new_img_path = os.path.join(dest_folder, os.path.basename(img_path))
                    shutil.copy(img_path, new_img_path)

                    # JSON 파일에서 이미지 정보 가져오기
                    if os.path.exists(json_path):
                        with open(json_path, 'r') as f:
                            data = json.load(f)
                        image_info = {
                            "id": image_id,
                            "file_name": os.path.basename(img_path),
                            "height": data["images"][0]["height"],  # 이미지 높이
                            "width": data["images"][0]["width"],   # 이미지 너비
                            "pl_code": data["images"][0]["pl_code"]  # 작물 코드
                        }
                        coco_dataset["images"].append(image_info)

                        # 주석(annotations) 정보 추가
                        for annotation in data["annotations"]:
                            annotation["id"] = self.annotation_id
                            annotation["image_id"] = image_id
                            self.annotation_id += 1
                            coco_dataset["annotations"].append(annotation)

                        # 카테고리 추가 (처음 한 번만 추가)
                        if not coco_dataset["categories"]:
                            coco_dataset["categories"] = data["categories"]

                    image_id += 1

        # annotations 파일을 저장합니다.
        with open(annotations_train_path, 'w') as f:
            json.dump(self.coco_train, f, indent=4)
        
        with open(annotations_val_path, 'w') as f:
            json.dump(self.coco_val, f, indent=4)


In [4]:
DATA_PATH = "C:/Users/user/Desktop/pj/noa/rose/새 폴더"
OUTPUT_PATH = "./datasets"
split_ratio_train = 0.8  # 훈련 데이터 비율
split_ratio_val = 0.2  # 검증 데이터 비율
data_splitter = DataSplitter(DATA_PATH, OUTPUT_PATH, split_ratio_train, split_ratio_val)
data_splitter.split_data()


이미지 분류 중: 0it [00:00, ?it/s]
이미지 선택 중:   0%|          | 0/1000 [00:00<?, ?it/s]
이미지 처리 중: 0it [00:00, ?it/s]
